In [ ]:
from pyincore import IncoreClient, FragilityService, MappingSet
from pyincore.analyses.meandamage import MeanDamage
from pyincore.analyses.buildingeconloss import BuildingEconLoss
from pyincore.analyses.buildingdamage import BuildingDamage

In [ ]:
# Connect to IN-CORE service
client = IncoreClient()

### Start with building damage

We are going to run three chained analyses, Building damage, Mean damage and finally Building economic loss. The output of one analysis is used as an input of the other. 

The example inventory is Shelby County, TN.

In [ ]:
# Create building damage
bldg_dmg = BuildingDamage(client)

In [ ]:
# Shelby County, TN RES3 building inventory
bldgs_id = "5a284f1fc7d30d13bc081a7e"
bldg_dmg.load_remote_input_dataset("buildings", bldgs_id)

# Load fragility mapping
fragility_service = FragilityService(client)

# Earthquake building mappings
mapping_set = MappingSet(fragility_service.get_mapping("5b47b350337d4a3629076f2c"))
bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

In [ ]:
# New madrid earthquake using Atkinson Boore 1995
bldg_dmg.set_parameter("hazard_type", "earthquake")
bldg_dmg.set_parameter("hazard_id", "5b902cb273c3371e1236b36b")
bldg_dmg.set_parameter("num_cpu", 1)

In [ ]:
bldg_dmg.set_parameter("result_name", "bldg_dmg_shelby")
bldg_dmg.set_parameter("fragility_key", "Non-Retrofit Fragility ID Code")

# Run building damage analysis
bldg_dmg.run_analysis()

# Retrieve result dataset
building_damage_result = bldg_dmg.get_output_dataset("ds_result")

### Mean damage

In [ ]:
# Create mean damage
md = MeanDamage(client)

# Chain damage output
md.set_input_dataset("damage", building_damage_result)

# Damage ratio file
md.load_remote_input_dataset("dmg_ratios", "5a284f2ec7d30d13bc08209a")

In [ ]:
md.set_parameter("result_name", "bldg_mean_dmg_shelby")
md.set_parameter("damage_interval_keys", ["DS_0", "DS_1", "DS_2", "DS_3"])
md.set_parameter("num_cpu", 1)

In [ ]:
# Run mean damage analysis
md.run_analysis()

# Retrieve result dataset
mean_damage_result = md.get_output_dataset('result')

### Building economic loss

In [ ]:
# Create building economic loss
bldg_econ_dmg = BuildingEconLoss(client)

# Set input datasets
bldg_econ_dmg.set_input_dataset("building_mean_dmg", mean_damage_result)
    
# Load Shelby building inventory
bldg_econ_dmg.load_remote_input_dataset("buildings", bldgs_id)

In [ ]:
# Load Occupancy Multiplier for structural damage
bldg_occupancy_mult_id = "61f3fc2f3ed7dd5c873bf9e7"
bldg_econ_dmg.load_remote_input_dataset("occupancy_multiplier", bldg_occupancy_mult_id)

In [ ]:
# set parameters
# Inflation factor. A user must supply the inflation percentage between
# building appraisal year and a year of interest (current, date of hazard etc.)
bldg_econ_dmg.set_parameter("inflation_factor", 2.5)

result_name = "shelby_bldg_econ_loss"
bldg_econ_dmg.set_parameter("result_name", result_name)

In [ ]:
# Run building economic loss analysis and show results
bldg_econ_dmg.run_analysis()


In [ ]:
# Retrieve result dataset
result = bldg_econ_dmg.get_output_dataset("result")

# Convert dataset to Pandas DataFrame
df = result.get_dataframe_from_csv(low_memory=False)

# Display top 5 rows of output data
df.head()